# svm

* 이진적 분류. 다중분류, 회귀분석 등 확장.
* hyperplane은 $w \cdot x - b=0$을 충족하는 점들의 집합 x를 말함.
* support vectors(hyperplane 상에 위치)간의 거리가 최대가 되도록.
* inner product: 커널을 계산할 때 사용.
* svm은 쉽게 확장
    * 선형 분리가능의 경우와 선형분리불가능의 차이는 상수C가 조건에 들어온 것.
    * 또, 비선형의 경우도 변수변환하면 커널을 그대로 사용함.
* 최적 커널의 heuristic한 결정

### 표기
* m: n_samples n: n_features
* $\left \langle x,y \right\rangle$: inner product, K (similarity function n_samples x n_samples vector)
* X: $m \times n$ vectors
* y: class labels, $y_i \in \{-1,1\}$
* $x_i$ 학습데이터의 i번째 벡터
* $\hat{y}$ predicted

### finding the maximum margin

* 학습데이터 ${x_i,y_i}\ where\ i=1 \ldots m$, $y_i \in y \{-1, +1\}, x_i \in \mathbb{R}^n$
  * 학습데이터 $ m \times n$벡터.
    행은 $1 \ldots m$,
    각 행은 $1 \ldots n$ 속성으로 구성 row (또는 column) vector ($\mathbb{R}^n$)
  * 클래스 y는 이진, -1 또는 1

|      | 속성1 | 속성2 | ... | 속성 n | 클래스 |
|------|------|------|------|------|-------|
|x1    |      |      |      |      |   1   |
|x2    |      |      |      |      |   1   |
|...   |      |      |      |      |   -1  |
|xm    |      |      |      |      |   1   |


### linear-separable (hard margin)

* H: $wx_i + b = 0$ 이를 hyperpance, decision boundary, decision surface라고도 함.
    * $H_1: wx_i + b \ge 0$이면 $y_i=1$
    * $H_2: wx_i + b \le 0$이면 $y_i=-1$
    * $y_i(wx_i + b) \ge 0\ \forall i$
    * $H_1$ ~ $H_2$ 거리는 $\frac{1}{\|w\|} \times 2$ (b가 1로 변경된 것은 정규화)
* 목적함수 max $\frac{2}{\|w\|}$ s.t. $y_i(wx_i + b)-1 \ge 0\ i=1,\cdots,n$
    * min문제로 바꾸면:
    * min $\frac{\|w\|^2}{2}$ s.t. $y_i(w \cdot x_i + b) \ge 1$
* 위 최적화 문제를 라그랑주 승수$\alpha$로 풀면 primal form:
    * $L_p=\frac{1}{2}\|w\|^2 - \sum_{i=1}^{n}{\alpha_i[y_i(w \cdot x_i + b)-1]}$
* 이를 w,b에 대해 편미분한 조건과 관련 조건으로 KT제한조건을 구하면:
    * $\frac{\partial L}{\partial w} = w - \sum \alpha_i y_i x_i = 0$
    * $\frac{\partial L}{\partial b} = -\sum y_i \alpha_i = 0$
    * $y_i(wx_i + b)-1 \ge 0\ i=1,\cdots,n$ (목적함수에서)
    * $\alpha_i \ge 0$
    * $\alpha_i(y_i(wx_i + b)-1) = 0\ i=1,\cdots,n$
    * 3,4번째 고려하면 $\alpha_i \ge 0$인 경우 support vectors ($H_1, H_2$에 있는 값). 그 외 $\alpha_i=0$
* 이 최적화는 quadratic programmming, smo로 풀 수 있슴.
* 라그랑주승수로 푼 편미분 결과를 대입, dual form $L_d$을 구하면 (Wolfe dual):
    * $L_d=\sum_{i=1}^n \alpha_i
        -\frac12 \sum_{i=1}^n \sum_{j=1}^n y_i y_j K(x_i, x_j) \alpha_i \alpha_j$
    * s.t $\sum_{i=1}^n y_i \alpha_i = 0, \alpha_i \ge 0, for i=1, 2, \cdots, n$

### linear: non-separable (soft margin)

* margin에 noisy데이터가 포함되어서 slack변수 $\xi$ ($\ge 0$)
* soft-margin SVM
    * minimize $\frac{\|w\|^2}{2}$
    * subject to $y_i(w \cdot x_i + b) \ge 1 - \xi$ 그리고 $\xi \ge 0$ for $i=1,2,\cdots,n$
* Lp 위 식에서 제약조건 2개를 Lagrange multipliers로 풀면 ($\alpha_i, \mu_i \ge 0$)
    * $L_p=\frac{1}{2}\|w\|^2 + C\sum_{i=1}^n\xi
    -\sum_{i=1}^{n}{\alpha_i[y_i(w \cdot x_i + b)-1+\xi_i]
    -\sum_{i=1}^n\mu_i\xi_i}$
* Kuhn-Tucker conditions
    * $\frac{\partial L}{\partial w} = w - \sum \alpha_i y_i x_i = 0$
    * $\frac{\partial L}{\partial b} = -\sum y_i \alpha_i = 0$
    * $\frac{\partial L}{\partial \xi} = C - \alpha_i - \mu_i = 0$
    * $y_i(wx_i + b)-1+\xi \ge 0\ i=1,\cdots,n$ (목적함수에서)
    * $\alpha_i \ge 0$
    * $\xi_i \ge 0$
    * $\mu_i \ge 0$
    * $\alpha_i(y_i(wx_i + b)-1+\xi_i) = 0$
    * $\mu_i \xi_i = 0,\ i=1,\cdots,n$
    * 3,7번째 고려하면 $\alpha_i \le C$
* Ld 조건을 수정하면
    * $L_d=\sum_{i=1}^n \alpha_i
        -\frac12 \sum_{i=1}^n \sum_{j=1}^n y_i y_j K(x_i, x_j) \alpha_i \alpha_j$
    * s.t $\sum_{i=1}^n y_i \alpha_i = 0,
        0 \le \alpha_i \le C,\ for\ i=1, 2, \cdots, n$
* C is an SVM hyperparameter and K(xi, xj) is the kernel function, both supplied by the user
* 위 KT 8번째에서, $\xi_i=0$일 경우:
    * $b = \frac{1}{y_i} - \sum_{i=1}^n y_i \alpha_i K(x_i,x_j)$
* 위 KT 2,7,8조건에서:
    * $\alpha_i=0 \Rightarrow y_i(w \cdot x_i + b) \ge 1\ and\ \xi_i=0$ 
    * $0 < \alpha_i < 0 \Rightarrow y_i(w \cdot x_i + b) = 1\ and\ \xi_i=0$ 
    * $\alpha_i=C \Rightarrow y_i(w \cdot x_i + b) \le 1\ and\ \xi_i \ge 0$ 

### non-linear

* 비선형 속성을 mapping하여, 기존 kernel K<x,x>을 그대로 사용함.
* Kernel은 inner product (즉 데이터를 서로 곱하여 단일 값이 나옴)을 유지하지만, mapping한 값으로 교체함.
이를 kernel trick (또는 kernel substitution)이라고 함.

### kernel tricks
* linear
* gaussian rbf
* poly

### Gaussian radial bias function

* $K(x,x') = \exp\left(-\frac{||x - x'||^2}{2\sigma^2}\right)$

* $||x - x'||^2$ squared Euclidean distance between the two feature vectors
* $\sigma$ a free parameter.
    간단하게 나타내면 $K(x,x') = \exp(\gamma||x - x'||^2)$, $\gamma = -\frac{1}{2\sigma^2}$ 


In [1]:
import numpy as np
_x=np.mat([[0,1],[2,3],[4,5],[6,7],[8,9]])
_m,_n=np.shape(_x)
_K=np.mat(np.zeros((_m,1))) #(m,1)
_sigma=1.3
for i in range(_m):
    _d=_x[i]-_x[0]
    _K[i]=_d*_d.T #(1,1) <-- (1,n)*(n,1)
_K=np.exp(_K/(-1*_sigma**2)) #(m,1)
print _K
print "linear일 경우에는\n",_x*_x[0].T

[[  1.00000000e+00]
 [  8.79362888e-03]
 [  5.97960549e-09]
 [  3.14422196e-19]
 [  1.27846878e-33]]
linear일 경우에는
[[1]
 [3]
 [5]
 [7]
 [9]]


In [1]:
import numpy as np
import sympy as sp
v0=np.array([0,1])
v1=np.array([2,3])
v2=np.array([4,5,6])

u0,u1,u2,u3,u4=sp.symbols('u:5')
u=np.array([u0,u1])

#v2mat=np.mat(v2)
print "multiply=",np.multiply(v1,v0),np.multiply(v1,u)
print "dot=",np.dot(v1,v0),np.dot(v1,u)
print "inner product=",np.inner(v1,v0),np.dot(v1,u)
print "outer product=\n",np.outer(v1,v0),"\n",np.outer(v1,u)

print "transpse=",v2.T

x=np.array([[1,2,3],[4,5,6],[7,8,9],[0,1,2],[2,3,4]])
print "x[0]={0}\nx[0].T={1}".format(x[0],x[0].T)
print "x[0]*x[0].T={0}".format(x*x[0].T)
print "inner product of array",np.inner(x,x[0]) #(5,3)


multiply= [0 3] [2*u0 3*u1]
dot= 3 2*u0 + 3*u1
inner product= 3 2*u0 + 3*u1
outer product=
[[0 2]
 [0 3]] 
[[2*u0 2*u1]
 [3*u0 3*u1]]
transpse= [4 5 6]
x[0]=[1 2 3]
x[0].T=[1 2 3]
x[0]*x[0].T=[[ 1  4  9]
 [ 4 10 18]
 [ 7 16 27]
 [ 0  2  6]
 [ 2  6 12]]
inner product of array [14 32 50  8 20]


## algorithm smo

* input: X(학습데이터), C, tol, max_passes
    * X: training dataset m(n_features) x n(n_samples)
    * C: regularization parameter
    * tol: numerical tolerance
    * max_passes
* output: alphas(Lagrange multipliers),b

절차
* initialize $\alpha_i=0, \forall i$
* while
    * for i=1...m
        * Ei계산
        * if not optimized
            * select j $\neq$ i
            * Ej계산
            * old alphas 저장
            * L,H bound 계산 (L==H이면 if문 빠져나감)
            * eta 계산 ($\eta \geq 0$ if문 빠져나감)
            * $\alpha_j$ 계산 (범위 제한)
            * $\alpha_i$ 계산
            * b 계산 (KKT 만족하도록)
            * 계산 후, LH 충족 검증
    * end for

알고리듬에 따른 개발
1. data read - 데이터 준비
2. Ei Ej 계산
3. L,H
4. alphas계산 및 검증
5. b 계산 및 검증


## 1. 데이터 준비

In [2]:
import pandas as pd
import numpy as np
import os

dir=os.getenv('HOME')+'/Code/git/else/machinelearninginaction/Ch06/'
filename=dir+'testSet.txt'
print "Data:\n"
!head testSet.txt

# 1.1 numpy reading
dt=np.dtype( {'names': ['x1','x2','c'], 'formats':['f','f','i'] } )
data=np.genfromtxt(filename, delimiter='\t', dtype=dt)
print "Data from numpy---\n",data[:5]
X=np.mat(zip(data['x1'],data['x2'])) #numpy matrix로 변환 (100,2)
y=np.mat(data['c']) #(1,100) array로 만들면 (100,)
yT=y.T #(100,1)
n_samples,n_features=X.shape #tuple size - len(data), len(data[0])
print "n_samples={0} n_features={1}\n{2}".format(n_samples,n_features,X[:5])

# 1.2 교재
#교재(p.108) dataMatrix와 동일. 
#import svmMLiA
#dataArr,labelArr=svmMLiA.loadDataSet('testSet.txt')
#X=np.mat(dataArr) #(100,2)
#y=np.mat(labelArr).transpose() #(100,1)
#yT=np.mat(labelArr).transpose()

print "dataMat X\n",X[:5]


Data:

head: testSet.txt: No such file or directory
Data from numpy---
[(3.542484998703003, 1.9773980379104614, -1)
 (3.0188961029052734, 2.5564160346984863, -1)
 (7.551509857177734, -1.5800299644470215, 1)
 (2.1149990558624268, -0.004466000013053417, -1)
 (8.127113342285156, 1.2743719816207886, 1)]
n_samples=100 n_features=2
[[  3.54248500e+00   1.97739804e+00]
 [  3.01889610e+00   2.55641603e+00]
 [  7.55150986e+00  -1.58002996e+00]
 [  2.11499906e+00  -4.46600001e-03]
 [  8.12711334e+00   1.27437198e+00]]
dataMat X
[[  3.54248500e+00   1.97739804e+00]
 [  3.01889610e+00   2.55641603e+00]
 [  7.55150986e+00  -1.58002996e+00]
 [  2.11499906e+00  -4.46600001e-03]
 [  8.12711334e+00   1.27437198e+00]]


## 2. Ei

* 오류E는 예측f(x)과 실제y와의 차이
* $E_i = f(X_i) - y_i$
* $f(x)=\sum_{i=1}^m \alpha_i y_i \langle x_i, x \rangle + b$

In [3]:
alphas=np.mat(np.zeros((n_samples,1)))
b=0
i=0
C=0.6
toler=0.001
max_iter=40

# 1) Ei 계산
fX=lambda i:float(np.multiply(alphas,yT).T*np.inner(X,X[i]))+b
alphaPairsChanged=0
fXi=fX(i)
Ei=fXi-float(yT[i]) #or Ei=fXi-float(y[0,i])
print "i={0} Ei={1} fXi={2} yi={2}".format(i,Ei,fXi,yT[i])

# 2) i가 optimize되었는지?
conditionA = (yT[i]*Ei < -toler) and (alphas[i] < C) #True
conditionB = (yT[i]*Ei > toler) and (alphas[i] > C) #False
print "if true, then unoptimized", conditionA or conditionB

# 3) optimize now! select j
#i와 j가 같지 않게!! 지금 i=0.
j=int(np.random.uniform(0,n_samples))
fXj=fX(j)
Ej=fXj-float(yT[j])
print "j={0} Ej={1} fXj={2} yj={3}".format(j,Ej,fXj,yT[j])

i=0 Ei=1.0 fXi=0.0 yi=0.0
if true, then unoptimized [[ True]]
j=77 Ej=-1.0 fXj=0.0 yj=[[1]]


## 3. LH

In [4]:
# 4) save old alphas
aj_old=alphas[j]
ai_old=alphas[i]

# 5) LH
if (yT[i] != yT[j]):
    L = max(0, alphas[j] - alphas[i]) #max(0,0)
    H = min(C, C + alphas[j] - alphas[i]) #min(C,C)
else:
    L = max(0, alphas[j] + alphas[i] - C) #max(0,-C)
    H = min(C, alphas[j] + alphas[i]) #min(C,0)
print "L={0} H={1}".format(L,H) #L=0, H=0.6

L=0 H=0.6


## 4. optimizing alpha

$\alpha_j := \alpha_j - \frac{y_j (E_i - E_j)}{\eta}$

$E_k=f(x_i) - y_i$

$\eta = 2 \langle x_i,x_j \rangle - \langle x_i,x_i \rangle - \langle x_j,x_j \rangle$

In [5]:
# 6) eta
eta= 2.0 * np.inner(X[i],X[j]) - np.inner(X[i],X[i]) - np.inner(X[j],X[j])
print "eta={0}".format(eta)

# 7) compute alphas
alphas[j] -= yT[j]*(Ei-Ej) / eta
alphas[i]=alphas[i]+yT[i]*yT[j]*(aj_old-alphas[j])
print "ai={0} aj={1} aj_old={2} ".format(alphas[i],alphas[j],aj_old)

eta=[[-23.73018646]]
ai=[[ 0.]] aj=[[ 0.08428084]] aj_old=[[ 0.08428084]] 


## 5. b threshold

In [6]:
# 8) compute b
aiyikii=yT[i]*(alphas[i]-ai_old)*np.inner(X[i],X[i])
ajyjkij=yT[j]*(alphas[j]-aj_old)*np.inner(X[i],X[j])
b1=b-Ei-aiyikii-ajyjkij

aiyikij=yT[i]*(alphas[i]-ai_old)*np.inner(X[i],X[j])
ajyjkjj=yT[j]*(alphas[j]-aj_old)*np.inner(X[j],X[j])

b2=b-Ej-aiyikij-ajyjkjj
print "b1={0} b2={1}".format(b1,b2)

b1=[[-1.]] b2=[[ 1.]]


#### Lagrange multiplier 라그랑주 승수법

* a strategy for finding the local maxima and minima of a function
subject to equality constraints.

* 예: the two-dimensional problem introduced above:
    * maximize f(x, y)
    * subject to g(x, y) = c
* 이럴 경우, 라그랑주 승수 $\lambda$를 사용하면 아래 식을 풀면 됨.
    * $\Lambda(x,y,\lambda) = f(x,y) + \lambda \cdot \Big(g(x,y)-c\Big)$

* 위키피디아 Example 1
http://en.wikipedia.org/wiki/Lagrange_multiplier



## case

### case 2 svm in korean 오일석 외 2008 (case20은 support vector가 아닌 데이터 추가.)
* K=[ 13.  11.  11.  17.]
* sum_aayyK=13.0*a0**2 - 22.0*a0*a1 + 17.0*a1**2
* Ld1=-6.5*a0**2 + 11.0*a0*a1 + a0 - 8.5*a1**2 + a1
* sum of ai*yi=0 const. a0 - a1
* [13.0*a0 11.0*a1]
* [-11.0*a0 -17.0*a1]
* {a0: 1/4, b: 1/2, a1: 1/4}
* 답 alphas=[1/4 1/4] b=1/2 w=[-1/2 1/2]


In [7]:
import numpy as np
import sympy as sp

# data
x2=np.array([[2,3],[4,1]])
y2=np.array([1,-1])
x20=np.array([[2,3],[4,1],[5,1]])
y20=np.array([1,-1,-1])
#sympy symbols
a0,a1,a2=sp.symbols('a:3')
b=sp.symbols('b')
a_2=np.array([a0,a1])

# CHANGE THIS!! (이하 모든 사례가 공통 계산 적용)
x=x2
y=y2
a=a_2

# 1. hyperplane
# 2. alphas 계산
m,n=x.shape
K = np.empty(shape=[0, m])
for i in range(m):
    ki=np.inner(x,x[i])
    print "k{0}={1}".format(i,ki)
    #K=np.append(K, [ki], axis=0)
    K=np.append(K,[ki])
print "K={0}".format(K)

print "sum of ai*yi=0 const.",sum(a*y) # a0=a1

for i in range(m):
    s=i*m
    e=s+m
    print a*y[i]*K[s:e]

# CHANGE THIS!!
case2=sp.solve([a0-a1,13*a0-11*a1+b-1,11*a0-17*a1+b+1],[a0,a1,b])
sol=case2
print sol

alphas = []
w=[]
for i in range(m):
    ai=sol[a[i]]
    alphas.append(ai)
    w.append(ai*y[i]*x[i])
alphas=np.array(alphas)
b=sol[b]
w=np.array(w)
w=w.sum(axis=0)
print "alphas={0} b={1} w={2}".format(alphas,b,w)


k0=[13 11]
k1=[11 17]
K=[ 13.  11.  11.  17.]
sum of ai*yi=0 const. a0 - a1
[13.0*a0 11.0*a1]
[-11.0*a0 -17.0*a1]
{a0: 1/4, b: 1/2, a1: 1/4}
alphas=[1/4 1/4] b=1/2 w=[-1/2 1/2]


In [8]:
import os
import numpy as np
dir=os.getenv('HOME')+'/Code/git/else/machinelearninginaction/Ch06/'
filename=dir+'testSetRBF.txt'
print "Data:\n"
os.chdir(dir)
!head testSetRBF.txt
import svmMLiA

# 1. data read
# Listing 6.8
dataArr,labelArr=svmMLiA.loadDataSet('testSetRBF.txt')
datMat=np.mat(dataArr)
labelMat=np.mat(labelArr)
labelMat=labelMat.transpose()

# 2. train - alphas, b 계산 (w는 하지 않음)
# smoSimple도 같은 기능. 그러나 10000회 실행에 30분 이상 실행함.
sigma=1.3
b,alphas=svmMLiA.smoP(dataArr,labelArr,200,0.0001,10000,('rbf',sigma))

Data:

-0.214824	0.662756	-1.000000
-0.061569	-0.091875	1.000000
0.406933	0.648055	-1.000000
0.223650	0.130142	1.000000
0.231317	0.766906	-1.000000
-0.748800	-0.531637	-1.000000
-0.557789	0.375797	-1.000000
0.207123	-0.019463	1.000000
0.286462	0.719470	-1.000000
0.195300	-0.179039	1.000000
fullSet, iter: 0 i:0, pairs changed 1
fullSet, iter: 0 i:1, pairs changed 1
fullSet, iter: 0 i:2, pairs changed 2
j not moving enough
fullSet, iter: 0 i:3, pairs changed 2
fullSet, iter: 0 i:4, pairs changed 2
fullSet, iter: 0 i:5, pairs changed 3
fullSet, iter: 0 i:6, pairs changed 4
fullSet, iter: 0 i:7, pairs changed 4
fullSet, iter: 0 i:8, pairs changed 5
fullSet, iter: 0 i:9, pairs changed 5
fullSet, iter: 0 i:10, pairs changed 5
j not moving enough
fullSet, iter: 0 i:11, pairs changed 5
fullSet, iter: 0 i:12, pairs changed 5
fullSet, iter: 0 i:13, pairs changed 5
fullSet, iter: 0 i:14, pairs changed 5
fullSet, iter: 0 i:15, pairs changed 6
fullSet, iter: 0 i:16, pairs changed 7
fullSet, iter: 0

In [9]:
# 3. sv 선별
svInd=np.nonzero(alphas.A>0)[0] #returns index for alphas>0
sVs=datMat[svInd]  #(m,n) support vectors
labelSV=labelMat[svInd] #(m,1)

#4. classify
# 1) listing 6.6 KernelTrans
kernelEval=svmMLiA.kernelTrans(sVs,datMat[0,:],('rbf', sigma)) #(m,1)
yhat_book = kernelEval.T*np.multiply(labelSV,alphas[svInd])+b #(1,1) <-- (1,m) * (m,1)
print "yhat_book=",yhat_book

# 2) my way
sv_alphas=alphas[svInd]
sv_y=labelSV
sv_X=sVs
sv_Xi=datMat[0]
m,n=np.shape(sv_X)
sv_K=np.mat(np.zeros((m,1)))
for i in range(m):
    d=sv_X[i]-sv_Xi
    sv_K[i]=d*d.T
sv_K=np.exp(sv_K/(-1*sigma**2))
sv_ay=np.multiply(sv_alphas,sv_y)
sv_ayK=sv_ay.T*sv_K #scalar결과
yhat = sv_ayK+b

def fX(b,alphas,y,X,Xi,sigma):
    m,n=np.shape(X)
    K=np.mat(np.zeros((m,1)))
    for i in range(m):
        d=X[i]-Xi
        K[i]=d*d.T
    K=np.exp(K/(-1*sigma**2))
    ay=np.multiply(alphas,y)
    ayK=ay.T*K
    fx=sum(ayK+b)
    return fx
yhat_fX=fX(b,alphas[svInd],labelSV,sVs,datMat[0,:],sigma)
print "yhat_fX=",yhat_fX
print "yhat={0} y={1} predicted? {2}".format(yhat,labelArr[0],np.sign(yhat) != np.sign(labelArr[0]))


yhat_book= [[-1.55569544]]
yhat_fX= [[-1.55569544]]
yhat=[[-1.55569544]] y=-1.0 predicted? [[False]]
